<a href="https://colab.research.google.com/github/arohanajit/disease-detector/blob/master/inasive-ductal-carcinoma/torch_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook
import shutil
train_on_gpu = torch.cuda.is_available()
print(train_on_gpu)

In [0]:
%%capture
if 'loader.py' not in os.listdir():
    !wget https://raw.githubusercontent.com/arohanajit/disease-detector/master/loader.py
import loader
if 'breast-histopathology-images.zip' not in os.listdir():
    loader.load_data()
    !kaggle datasets download -d paultimothymooney/breast-histopathology-images
    !unzip breast-histopathology-images.zip
    for i in os.listdir():
        try:
            a = int(i[0])
            shutil.rmtree(i,ignore_errors=True)
        except:
            continue

In [0]:
from glob import glob
imagePatches = glob('IDC_regular_ps50_idx5/**/*.png', recursive=True)
print(len(imagePatches))

In [0]:
data_df = pd.DataFrame(index=np.arange(0, len(imagePatches)), columns=["pid", "img", "label"])
base_dir = 'IDC_regular_ps50_idx5/'
count = 0
for i in os.listdir(base_dir):
    pid = i
    patient_path = base_dir + pid
    lbl = [0,1]
    for j in lbl:
        img_path = patient_path + '/' + str(j) + '/'
        for k in os.listdir(img_path):
            data_df.iloc[count]["pid"] = pid
            data_df.iloc[count]["img"] = img_path+'/'+k
            data_df.iloc[count]["label"] = j
            if count%50000==0:
                print("{} Done".format(count))
            count+=1


In [0]:
print(data_df.shape)
data_df.head()